In [1]:

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import SelectorGroupChat
from autogen_agentchat.conditions import MaxMessageTermination
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.tools import FunctionTool
from dotenv import load_dotenv
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
import os
import asyncio
load_dotenv()
gemini_api_key=os.getenv('GEMINI_API_KEY')
print("Gemini key loaded:", gemini_api_key is not None)

Gemini key loaded: True


In [2]:
async def add_numbers(a: int, b: int) -> int:
    """Adds two numbers and returns the result."""
    return a + b
addition_tool=FunctionTool(add_numbers,description="useful for addition operations.")

In [3]:
addition_tool.schema

{'name': 'add_numbers',
 'description': 'useful for addition operations.',
 'parameters': {'type': 'object',
  'properties': {'a': {'description': 'a', 'title': 'A', 'type': 'integer'},
   'b': {'description': 'b', 'title': 'B', 'type': 'integer'}},
  'required': ['a', 'b'],
  'additionalProperties': False},
 'strict': False}

In [4]:
model_client=OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=gemini_api_key,
        )

In [13]:
from autogen_core.models import AssistantMessage, FunctionExecutionResult, FunctionExecutionResultMessage, UserMessage
user=UserMessage(content="add 100 with 90",source="user")
cancellation_token = CancellationToken()


In [8]:
result=await model_client.create(messages=[user],tools=[addition_tool],cancellation_token=cancellation_token)

In [12]:
result.content

[FunctionCall(id='', arguments='{"b":90,"a":100}', name='add_numbers')]

In [18]:
import json

In [19]:
assert isinstance(result.content, list)
arguments = json.loads(result.content[0].arguments)  
tool_result = await addition_tool.run_json(arguments, cancellation_token)
tool_result_str = addition_tool.return_value_as_string(tool_result)
tool_result_str


'190'

In [21]:

exec_result = FunctionExecutionResult(
    call_id=result.content[0].id, 
    content=tool_result_str,
    is_error=False,
    name=addition_tool.name,
)


messages = [
    user,
    AssistantMessage(content=result.content, source="assistant"),   
    FunctionExecutionResultMessage(content=[exec_result]),  
]
create_result = await model_client.create(messages=messages, cancellation_token=cancellation_token)  # 
print(create_result.content)
await model_client.close()


100 added to 90 is 190.


In [32]:
agent=AssistantAgent(
    model_client=OpenAIChatCompletionClient(
            model="gemini-2.5-flash",
            api_key=gemini_api_key,
        ),
    name="jarvis",
    tools=[add_numbers],
    system_message="you are a helpfull assistant, goal is to give answer to the user use the tool when needed."

)

In [33]:
await Console(agent.run_stream(task="add 100 with 90."))

---------- TextMessage (user) ----------
add 100 with 90.
---------- ToolCallRequestEvent (jarvis) ----------
[FunctionCall(id='', arguments='{"b":90,"a":100}', name='add_numbers')]
---------- ToolCallExecutionEvent (jarvis) ----------
[FunctionExecutionResult(content='190', name='add_numbers', call_id='', is_error=False)]
---------- ToolCallSummaryMessage (jarvis) ----------
190


TaskResult(messages=[TextMessage(id='365a41b4-ab10-4664-8339-3b74cd0a177e', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 7, 12, 23, 7, 727025, tzinfo=datetime.timezone.utc), content='add 100 with 90.', type='TextMessage'), ToolCallRequestEvent(id='8f38aba9-5f61-4d20-ba19-3644c3aa20cd', source='jarvis', models_usage=RequestUsage(prompt_tokens=84, completion_tokens=23), metadata={}, created_at=datetime.datetime(2025, 9, 7, 12, 23, 9, 58442, tzinfo=datetime.timezone.utc), content=[FunctionCall(id='', arguments='{"b":90,"a":100}', name='add_numbers')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(id='b95dda06-c99d-4c8d-8580-d4522748542c', source='jarvis', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 9, 7, 12, 23, 9, 68868, tzinfo=datetime.timezone.utc), content=[FunctionExecutionResult(content='190', name='add_numbers', call_id='', is_error=False)], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(id='a932d0ff-edcd